In [1]:
import math
import csv
from numpy import genfromtxt
import numpy as np
import h5py
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.python.framework import ops
from tf_utils import load_dataset, random_mini_batches, convert_to_one_hot, predict
import pandas as pd

%matplotlib inline
np.random.seed(1)

/srv/conda/envs/notebook/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/srv/conda/envs/notebook/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/srv/conda/envs/notebook/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/srv/conda/envs/notebook/lib/python3.7/site-packages/tensorflow/python/frame

In [2]:
# the original dataset as numpy matrix 
XY_orig=genfromtxt('daily_churn_pred_3_1.csv', delimiter=',') 
XY_orig=np.matrix(XY_orig)   

# taking out the X portion
X_orig=XY_orig[1:80001, 1:12].T   
X_orig_max = X_orig.max(1)          
X_orig=X_orig/X_orig_max
print ("X_orig shape: " + str(X_orig.shape))

# taking out the Y portion
Y_orig=XY_orig[1:80001, 12].T       

# training set
X_train=X_orig[:, 0:75000]          
Y_train=Y_orig[:, 0:75000]          

# test set
X_test=X_orig[:, 75000:80000]
Y_test=Y_orig[:, 75000:80000]       

# convert all examples to numpy arrays
X_train=np.array(X_train)   
X_train=np.nan_to_num(X_train, copy=True, nan=0.0, posinf=None, neginf=None)

X_test=np.array(X_test)
X_test=np.nan_to_num(X_test, copy=True, nan=0.0, posinf=None, neginf=None)

Y_train=np.array(Y_train)
Y_test=np.array(Y_test)

# convert to one-hot representations
Y_train=Y_train.astype(int)         
Y_test=Y_test.astype(int)
Y_train=convert_to_one_hot(Y_train, 2)
Y_test=convert_to_one_hot(Y_test, 2)

# printing out dimensions
print ("number of training examples = " + str(X_train.shape[1]))      
print ("number of test examples = " + str(X_test.shape[1]))
print ("X_train shape: " + str(X_train.shape))
print ("Y_train shape: " + str(Y_train.shape))
print ("X_test shape: " + str(X_test.shape))
print ("Y_test shape: " + str(Y_test.shape))

# printing out details
print("NaNs exist in training set: " + str(np.isnan(X_train).any()))
print("NaNs exist in test set: " + str(np.isnan(X_test).any()))

X_orig shape: (11, 80000)
number of training examples = 75000
number of test examples = 5000
X_train shape: (11, 75000)
Y_train shape: (2, 75000)
X_test shape: (11, 5000)
Y_test shape: (2, 5000)
NaNs exist in training set: False
NaNs exist in test set: False


In [3]:
# the real dataset as a numpy matrix 
X_real=genfromtxt('daily_churn_pred_2_full.csv', delimiter=',') 
X_real=np.matrix(X_real)

# taking out the X portion and normalizing the matrix
X_real=X_real[1:, 1:12].T   
X_real_max=X_real.max(1)          
X_real=X_real/X_real_max

# convert all examples to numpy arrays for replacing NaNs
X_real=np.array(X_real)   
X_real=np.nan_to_num(X_real, copy=True, nan=0.0, posinf=None, neginf=None)

# printing out dimensions
print ("number of real examples = " + str(X_real.shape[1]))      
print ("X_real shape: " + str(X_real.shape))

# printing out details
print("NaNs exist in training set: " + str(np.isnan(X_real).any()))

# taking out the mobile numbers 
df_mobile_nos=pd.read_csv("daily_churn_pred_2_full.csv") 
mobile_nos=list(df_mobile_nos["mobile_no"])

# users with differently predicted retentions
pred_0=[]
pred_1=[]

number of real examples = 141678
X_real shape: (11, 141678)
NaNs exist in training set: False


In [4]:
def create_placeholders(n_x, n_y):

    X = tf.placeholder(tf.float32, [n_x, None])
    Y = tf.placeholder(tf.float32, [n_y, None])
    
    return X, Y

In [5]:
def initialize_parameters():
    
    # keeping results consistent
    tf.set_random_seed(1)                   
        
    # defining the neural network's architecture
    W1 = tf.get_variable("W1", [20, 11], initializer = tf.contrib.layers.xavier_initializer(seed = 1))
    b1 = tf.get_variable("b1", [20, 1], initializer = tf.zeros_initializer())
    
    W2 = tf.get_variable("W2", [25, 20], initializer = tf.contrib.layers.xavier_initializer(seed = 1))
    b2 = tf.get_variable("b2", [25, 1], initializer = tf.zeros_initializer())
    
    W3 = tf.get_variable("W3", [25, 25], initializer = tf.contrib.layers.xavier_initializer(seed = 1))
    b3 = tf.get_variable("b3", [25, 1], initializer = tf.zeros_initializer())
    
    W4 = tf.get_variable("W4", [25, 25], initializer = tf.contrib.layers.xavier_initializer(seed = 1))
    b4 = tf.get_variable("b4", [25, 1], initializer = tf.zeros_initializer())
    
    W5 = tf.get_variable("W5", [20, 25], initializer = tf.contrib.layers.xavier_initializer(seed = 1))
    b5 = tf.get_variable("b5", [20, 1], initializer = tf.zeros_initializer())
    
    W6 = tf.get_variable("W6", [2, 20], initializer = tf.contrib.layers.xavier_initializer(seed = 1))
    b6 = tf.get_variable("b6", [2, 1], initializer = tf.zeros_initializer())

    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2,
                  "W3": W3,
                  "b3": b3,
                  "W4": W4,
                  "b4": b4,
                  "W5": W5,
                  "b5": b5,
                  "W6": W6,
                  "b6": b6}
    
    return parameters

In [6]:
def forward_propagation(X, parameters):
    
    # retrieving the parameters from the dictionary 'parameters' 
    W1 = parameters['W1']
    b1 = parameters['b1']
    W2 = parameters['W2']
    b2 = parameters['b2']
    W3 = parameters['W3']
    b3 = parameters['b3']
    W4 = parameters['W4']
    b4 = parameters['b4']
    W5 = parameters['W5']
    b5 = parameters['b5']
    W6 = parameters['W6']
    b6 = parameters['b6']
                                         
    Z1 = tf.add(tf.matmul(W1, X), b1)                                              
    A1 = tf.nn.relu(Z1)   
    
    Z2 = tf.add(tf.matmul(W2, A1), b2)                                             
    A2 = tf.nn.relu(Z2)  
    
    Z3 = tf.add(tf.matmul(W3, A2), b3) 
    A3 = tf.nn.relu(Z3)
    
    Z4 = tf.add(tf.matmul(W4, A3), b4)
    A4 = tf.nn.relu(Z4)
    
    Z5 = tf.add(tf.matmul(W5, A4), b5)
    A5 = tf.nn.relu(Z5)
    
    Z6 = tf.add(tf.matmul(W6, A5), b6)
    
    return Z6

In [7]:
def compute_cost(Z6, Y):
    
    # to fit the tensorflow requirement for tf.nn.softmax_cross_entropy_with_logits(...,...)
    logits = tf.transpose(Z6)
    labels = tf.transpose(Y)
    
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=labels))
    
    return cost

In [8]:
def model(X_train, Y_train, X_test, Y_test, learning_rate = 0.01,
          num_epochs = 500, minibatch_size = 64, print_cost = True):
    
    ops.reset_default_graph()                         # to be able to rerun the model without overwriting tf variables
    tf.set_random_seed(1)                             # to keep consistent results
    seed = 3                                          # to keep consistent results
    (n_x, m) = X_train.shape                          # n_x: input size, m : number of examples in the train set
    n_y = Y_train.shape[0]                            # n_y: output size
    costs = []                                        # to keep track of the cost
    
    # creating Placeholders of shape (n_x, n_y)
    X, Y = create_placeholders(n_x, n_y)

    # initializing parameters
    parameters = initialize_parameters()
    
    # forward propagation: building the forward propagation in the tensorflow graph
    Z6 = forward_propagation(X, parameters)
    
    # cost function: adding cost function to tensorflow graph
    cost = compute_cost(Z6, Y)
    
    # backpropagation: defining the tensorflow AdamOptimizer
    optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(cost)
    
    # initializing all the variables
    init = tf.global_variables_initializer()

    # starting the session to compute the tensorflow graph
    with tf.Session() as sess:
        
        # running the initialization
        sess.run(init)
        
        # training loop
        for epoch in range(num_epochs):

            epoch_cost = 0.                       
            num_minibatches = int(m / minibatch_size) # number of minibatches of size minibatch_size in the train set
            seed = seed + 1
            minibatches = random_mini_batches(X_train, Y_train, minibatch_size, seed)

            for minibatch in minibatches:

                # selecting a minibatch
                (minibatch_X, minibatch_Y) = minibatch
                
                _ , minibatch_cost = sess.run([optimizer, cost], feed_dict={X: minibatch_X, Y: minibatch_Y})
                
                epoch_cost += minibatch_cost / num_minibatches

            # printing the cost every epoch
            if print_cost == True and epoch % 100 == 0:
                print ("Cost after epoch %i: %f" % (epoch, epoch_cost))
            if print_cost == True and epoch % 5 == 0:
                costs.append(epoch_cost)

        # saving the parameters in a variable
        parameters = sess.run(parameters)
        
        print ("Parameters have been trained!")
        print()

        # accuracy
        correct_prediction = tf.equal(tf.argmax(Z6), tf.argmax(Y))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
        
        print ("Train Accuracy:", accuracy.eval({X: X_train, Y: Y_train}))
        print ("Test Accuracy:", accuracy.eval({X: X_test, Y: Y_test}))
        print()
        
        # confusion matrix
        predicted0_actual0 = tf.reduce_sum(tf.cast(tf.logical_and(tf.equal(tf.argmax(Z6), 0), tf.equal(tf.argmax(Y), 0)), "float"))
        predicted0_actual1 = tf.reduce_sum(tf.cast(tf.logical_and(tf.equal(tf.argmax(Z6), 0), tf.equal(tf.argmax(Y), 1)), "float"))
        predicted1_actual0 = tf.reduce_sum(tf.cast(tf.logical_and(tf.equal(tf.argmax(Z6), 1), tf.equal(tf.argmax(Y), 0)), "float"))
        predicted1_actual1 = tf.reduce_sum(tf.cast(tf.logical_and(tf.equal(tf.argmax(Z6), 1), tf.equal(tf.argmax(Y), 1)), "float"))

        print("True negatives:", predicted0_actual0.eval({X: X_test, Y: Y_test}))
        print("False negatives:", predicted0_actual1.eval({X: X_test, Y: Y_test}))
        print("False positives:", predicted1_actual0.eval({X: X_test, Y: Y_test}))
        print("True positives:", predicted1_actual1.eval({X: X_test, Y: Y_test}))
        print()

        # precision, recall, f1-score
        recall=tf.divide(predicted1_actual1, tf.add_n([predicted1_actual1, predicted0_actual1]))
        precision=tf.divide(predicted1_actual1, tf.add_n([predicted1_actual1, predicted1_actual0]))

        metric_train_prec=precision.eval({X: X_train, Y: Y_train})
        print("Train Precision:", metric_train_prec)
        metric_test_prec=precision.eval({X: X_test, Y: Y_test})
        print("Test Precision:", metric_test_prec)
        
        metric_train_rec=recall.eval({X: X_train, Y: Y_train})
        print("Train Recall:", metric_train_rec)
        metric_test_rec=recall.eval({X: X_test, Y: Y_test})
        print("Test Recall:", metric_test_rec)
        print()
        
        metric_train_f1=(2*metric_train_prec*metric_train_rec)/(metric_train_prec+metric_train_rec)
        print("Train f1:", metric_train_f1)
        metric_test_f1=(2*metric_test_prec*metric_test_rec)/(metric_test_prec+metric_test_rec)
        print("Test f1:", metric_test_f1)
        print()
        
        # predictions against test-set
        predicted0=tf.equal(tf.argmax(Z6), 0)
        predicted1=tf.equal(tf.argmax(Z6), 1)
        
        predicted_zeros=list(predicted0.eval({X: X_real}))
        len_predicted_zeros=len(predicted_zeros)
        print(len_predicted_zeros)
        print(len(mobile_nos))
        for i in range(0, len_predicted_zeros):
            if(predicted_zeros[i]==True): pred_0.append(mobile_nos[i])
                
        predicted_ones=list(predicted1.eval({X: X_real}))
        len_predicted_ones=len(predicted_ones)
        print(len_predicted_ones)
        for i in range(0, len_predicted_ones):
            if(predicted_ones[i]==True): pred_1.append(mobile_nos[i])
        
        return parameters

In [9]:
parameters = model(X_train, Y_train, X_test, Y_test)


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

Cost after epoch 0: 0.480543
Cost after epoch 100: 0.462995
Cost after epoch 200: 0.463432
Cost after epoch 300: 0.463705
Cost after epoch 400: 0.464497
Parameters have been trained!

Train Accuracy: 0.76713336
Test Accuracy: 0.7622

True negatives: 3396.0
False negatives: 845.0
False positives: 344.0
True positives: 415.0

Train Precision: 0.5585131
Test Precision: 0.54677206
Train Recall: 0.32470652
Test Recall: 0.32936507

Train f1: 0.4106630996488083
Test f1: 0.41109459598269155

141678
14167

In [10]:
pred_1=['0'+str(x) for x in pred_1]
print(str(len(pred_1)) + " users predicted not to return are:\n")
print(pred_1)

# converting to DataFrame to csv
dict = {'pred_churn': pred_1} 
df = pd.DataFrame(dict)
df.to_csv("pred_churn.csv", index=False)

19045 users predicted not to return are:

['01967116490', '01830183824', '01828286934', '01787987916', '01718682320', '01776339281', '01705105381', '01712940750', '01711368138', '01703116450', '01721257074', '01921704436', '01918080888', '01969449077', '01316607795', '01964300346', '01762799783', '01908091916', '01777753733', '01701760077', '01714333050', '01974488245', '01769090699', '01780442233', '01917203923', '01720584170', '01994753669', '01886611224', '01678862353', '01939429339', '01727660027', '01914112607', '01770036007', '01824829283', '01759370552', '01817056765', '01964643314', '01867427754', '01313393980', '01313192706', '01815233281', '01629383584', '01936673693', '01762968125', '01868871288', '01872602871', '01714003251', '01850123179', '01818295233', '01305667233', '01580979757', '01785838402', '01748342110', '01713980160', '01723487261', '01815853049', '01856815135', '01911512538', '01718897348', '01732925481', '01857462903', '01923803298', '01977388547', '01737660219

In [11]:
pred_0=['0'+str(x) for x in pred_0]
print(str(len(pred_0)) + " users predicted to return are:\n")
print(pred_0)

122633 users predicted to return are:

['01910022514', '01781368495', '01720613765', '01993039553', '01916196666', '01836311027', '01716325354', '01875190620', '01889195156', '01721510712', '01717697347', '01710462025', '01815929191', '01834942171', '01752879338', '01737145221', '01757579112', '01775696927', '01757839832', '01713682411', '01829664601', '01741727703', '01779473786', '01815658786', '01741047931', '01721479234', '01884027599', '01625999115', '01690027451', '01813346234', '01780786686', '01611600674', '01934334418', '01991214026', '01609116000', '01771289027', '01714474291', '01744567217', '01737223832', '01741605314', '01715034582', '01716311556', '01315096507', '01731359685', '01725296777', '01311218269', '01902741024', '01847023385', '01761206091', '01710072433', '01617971890', '01925780590', '01882730508', '01850563626', '01301370337', '01727959824', '01924731346', '01615608548', '01991412426', '01819632242', '01311750008', '01737894681', '01915371595', '01857963008', 